In [10]:
import requests
# ! pip install pgeocode
import pgeocode
import numpy as np
import pandas as pd
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import json

In [11]:
CLIENT_ID = 'NQJXQGYTHHRFUJNUELLI5CYLPTB34HFYM41WDYS2J1O1WVWQ' # your Foursquare ID
CLIENT_SECRET = 'BWTLRL41DWVXGE33DUPTXX2E4RLNJQQBFJEKMOUZQBHE3CNW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
#put your census API key here
apiKey = "23b6cb9e00cfc26100d2f7ba7cdee5530d70ed58"

In [12]:
nomi = pgeocode.Nominatim('us')
zipcodes = [str(i) for i in np.arange(43001,46000)]
a = nomi.query_postal_code(zipcodes)
a = a[a['state_name']=='Ohio']
a = a[['postal_code','place_name','county_name','latitude','longitude']]

In [27]:
df = a[a.county_name.str.contains('Clark',case=False) | a.county_name.str.contains('Miami',case=False) | a.county_name.str.contains('Greene',case=False) | a.county_name.str.contains('Montgomery',case=False)]
df.head()
# df['postal_code'].drop_duplicates()

,postal_code,place_name,county_name,latitude,longitude
9,43010,Catawba,Clark,39.9991,-83.6222
2300,45301,Alpha,Greene,39.7117,-84.0233
2304,45305,Bellbrook,Greene,39.6402,-84.0824
2306,45307,Bowersville,Greene,39.5806,-83.7249
2307,45308,Bradford,Miami,40.1286,-84.4293


In [28]:
OHZips=list(df['postal_code'])
OHZips = ','.join(OHZips)

#construct the API call we will use
baseAPI = "https://api.census.gov/data/2018/acs/acs5?key=%s&get=B01003_001E&for=zip%%20code%%20tabulation%%20area:%s" 
calledAPI = baseAPI % (apiKey, OHZips)

#call the API and collect the response
response = requests.get(calledAPI)

#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]

#flip the order of the response from [population, zipcode] -> [zipcode, population]
formattedResponse = [item[::-1] for item in formattedResponse]

#store the response in a dataframe
OHZipPopulations = pd.DataFrame(columns=['zipcode', 'population'], data=formattedResponse)
OHZipPopulations.head()

,zipcode,population
0,43010,260
1,45384,2053
2,45319,378
3,45361,328
4,45372,319


In [29]:
df = df.join(OHZipPopulations.set_index('zipcode'), on='postal_code')
df.dropna(inplace=True)
print(df.shape)
df.tail()

(74, 6)


,postal_code,place_name,county_name,latitude,longitude,population
2501,45502,Springfield,Clark,39.9242,-83.8088,15779
2502,45503,Springfield,Clark,39.9528,-83.7804,32363
2503,45504,Springfield,Clark,39.9408,-83.8343,17639
2504,45505,Springfield,Clark,39.9106,-83.7856,19919
2505,45506,Springfield,Clark,39.9104,-83.8275,13868


In [16]:
LIMIT = 100
radius = 2000
def getNearbyVenues(names, latitudes, longitudes, radius=3200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
Dayton_venues = getNearbyVenues(names=df['postal_code'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )
Dayton_venues.head()

43010
45301
45305
45307
45308
45309
45312
45314
45315
45316
45317
45318
45319
45322
45323
45324
45325
45326
45327
45335
45337
45339
45341
45342
45344
45345
45349
45354
45356
45359
45361
45368
45369
45370
45371
45372
45373
45377
45383
45384
45385
45387
45402
45403
45404
45405
45406
45409
45410
45414
45415
45416
45417
45419
45420
45424
45426
45428
45429
45430
45431
45432
45433
45434
45439
45440
45449
45458
45459
45502
45503
45504
45505
45506


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,43010,39.9991,-83.6222,Yuengling Acres,39.986082,-83.590175,Pub
1,45301,39.7117,-84.0233,Ritter's Frozen Custard,39.717243,-84.029283,Ice Cream Shop
2,45301,39.7117,-84.0233,Rotary Park,39.716949,-84.016187,Baseball Field
3,45301,39.7117,-84.0233,The Narrows Reserve,39.690426,-84.029462,Nature Preserve
4,45301,39.7117,-84.0233,Kroger Marketplace,39.725390,-84.054144,Supermarket


In [20]:
Dayton_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
43010,1,1,1,1,1,1
45301,29,29,29,29,29,29
45305,72,72,72,72,72,72
45307,4,4,4,4,4,4
45308,6,6,6,6,6,6
...,...,...,...,...,...,...
45502,100,100,100,100,100,100
45503,36,36,36,36,36,36
45504,100,100,100,100,100,100


In [21]:
print('There are {} uniques categories.'.format(len(Dayton_venues['Venue Category'].unique())))

There are 260 uniques categories.


In [37]:
# one hot encoding
dayton_onehot = pd.get_dummies(Dayton_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dayton_onehot['Neighborhood'] = Dayton_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dayton_onehot.columns[-1]] + list(dayton_onehot.columns[:-1])
dayton_onehot = dayton_onehot[fixed_columns]

dayton_grouped = dayton_onehot.groupby('Neighborhood').mean().reset_index()
dayton_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Airport,Airport Service,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,43010,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
1,45301,0.0,0.00,0.0,0.0,0.0,0.034483,0.0,0.00,0.0,...,0.0,0.000000,0.034483,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
2,45305,0.0,0.00,0.0,0.0,0.0,0.027778,0.0,0.00,0.0,...,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.00
3,45307,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
4,45308,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,45502,0.0,0.01,0.0,0.0,0.0,0.030000,0.0,0.00,0.0,...,0.0,0.000000,0.010000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01
69,45503,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.000000,0.027778,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
70,45504,0.0,0.01,0.0,0.0,0.0,0.050000,0.0,0.01,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.020000,0.01
71,45505,0.0,0.00,0.0,0.0,0.0,0.011494,0.0,0.00,0.0,...,0.0,0.000000,0.011494,0.0,0.0,0.0,0.0,0.0,0.000000,0.00


In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dayton_grouped['Neighborhood']

for ind in np.arange(dayton_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dayton_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43010,Pub,Women's Store,Event Space,Fountain,Football Stadium,Food Truck,Food Stand,Food & Drink Shop,Food,Fondue Restaurant
1,45301,Intersection,Nature Preserve,Pizza Place,Soccer Field,Other Great Outdoors,Shipping Store,Basketball Court,Supermarket,Sandwich Place,Coffee Shop
2,45305,Bank,Sandwich Place,Park,Grocery Store,Pizza Place,Coffee Shop,Hardware Store,Gas Station,Mexican Restaurant,Supermarket
3,45307,Farm,Trail,Rest Area,Flower Shop,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fondue Restaurant
4,45308,Grocery Store,Construction & Landscaping,Gas Station,Discount Store,Pizza Place,Food,Women's Store,Flower Shop,Farmers Market,Fast Food Restaurant


In [39]:
dayton_grouped = dayton_grouped.join(OHZipPopulations.set_index('zipcode'), on='Neighborhood')
dayton_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Airport,Airport Service,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,population
0,43010,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,260
1,45301,0.0,0.00,0.0,0.0,0.0,0.034483,0.0,0.00,0.0,...,0.000000,0.034483,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,130
2,45305,0.0,0.00,0.0,0.0,0.0,0.027778,0.0,0.00,0.0,...,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.00,11161
3,45307,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,330
4,45308,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,4139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,45502,0.0,0.01,0.0,0.0,0.0,0.030000,0.0,0.00,0.0,...,0.000000,0.010000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,15779
69,45503,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.000000,0.027778,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,32363
70,45504,0.0,0.01,0.0,0.0,0.0,0.050000,0.0,0.01,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.020000,0.01,17639
71,45505,0.0,0.00,0.0,0.0,0.0,0.011494,0.0,0.00,0.0,...,0.000000,0.011494,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,19919


In [40]:
# set number of clusters
kclusters = 5

dayton_grouped_clustering = dayton_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dayton_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 4, 4, 2, 4, 0, 4, 4], dtype=int32)

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dayton_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dayton_merged = dayton_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='postal_code')

dayton_merged.head() # check the last columns!

,postal_code,place_name,county_name,latitude,longitude,population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,43010,Catawba,Clark,39.9991,-83.6222,260,4.0,Pub,Women's Store,Event Space,Fountain,Football Stadium,Food Truck,Food Stand,Food & Drink Shop,Food,Fondue Restaurant
2300,45301,Alpha,Greene,39.7117,-84.0233,130,4.0,Intersection,Nature Preserve,Pizza Place,Soccer Field,Other Great Outdoors,Shipping Store,Basketball Court,Supermarket,Sandwich Place,Coffee Shop
2304,45305,Bellbrook,Greene,39.6402,-84.0824,11161,0.0,Bank,Sandwich Place,Park,Grocery Store,Pizza Place,Coffee Shop,Hardware Store,Gas Station,Mexican Restaurant,Supermarket
2306,45307,Bowersville,Greene,39.5806,-83.7249,330,4.0,Farm,Trail,Rest Area,Flower Shop,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fondue Restaurant
2307,45308,Bradford,Miami,40.1286,-84.4293,4139,4.0,Grocery Store,Construction & Landscaping,Gas Station,Discount Store,Pizza Place,Food,Women's Store,Flower Shop,Farmers Market,Fast Food Restaurant


In [43]:
dayton_merged.dropna(inplace=True)

In [44]:
# create map
map_clusters = folium.Map(location=[dayton_merged['latitude'].mean(), dayton_merged['longitude'].mean()], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dayton_merged['latitude'], dayton_merged['longitude'], dayton_merged['place_name'], dayton_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters